### Bibliotecas usadas

In [32]:
%pylab inline
import glob
import pandas as pd
import math
import dateutil

Populating the interactive namespace from numpy and matplotlib


### Criação de um DataFrame com as informações dos CSV

In [38]:
path = '../csv/original/'
filenames = glob.glob(path + "/*.csv")

list_ = []
frame = pd.DataFrame()

# Para cada um dos csv
for filename in filenames:
    # Leia o CSV
    df = pd.read_csv(filename, index_col = False)
    # Coloque na lista de CSV
    list_.append(df)

frame = pd.concat(list_)

# Dataframe
df_clean = frame

df_clean.head()

,Data Emissão,OB Origem,NE Origem,No Processo,Credor,Nome Credor,Finalidade,Fonte de Recurso,Natureza Detalhada,Nome Natureza Detalhada,UG Executora,Nome UG Executora,Mês Emissão,Pago
0,02/10/2017,2017OB800355,2017NE800036,23106025764/2014-57,00.606.194/0001-73,KOZCOE ENGENHARIA LTDA,PAGAMENTO NOTA FISCAL NF.003.327 - KOZCOE ENGE...,0250154079,3390.39.10,LOCACAO DE IMOVEIS,154079,CENTRO DE SEL.E DE\r\nPROMOCAO DE EVENTOS,10,"39.667,80"
1,02/10/2017,2017OB806258,2017NE000016,23106118433/2017-94,790.113.491-72,REGINA THELMA MEDEIROS\r\nMESQUITA DE ANDRADE,PAGAMENTO DA FOLHA- 09/2017-702 // AUXILIO A\r...,0250022135,3390.20.01,AUXILIO A PESQUISADORES,154019,CENTRO DE APOIO AO\r\nDESENVOLV. TECNOLOGICO\r...,10,"1.810,00"
2,02/10/2017,2017OB806259,2017NE000015,23106.118156/2017-10,376.777.641-34,ALESSANDRO DELON\r\nCAMARGO MESQUITA,PAGAMENTO DA FOLHA- 09/2017-715 // AUXILIO A\r...,0250022135,3390.20.01,AUXILIO A PESQUISADORES,154019,CENTRO DE APOIO AO\r\nDESENVOLV. TECNOLOGICO\r...,10,"3.500,00"
3,02/10/2017,2017OB806260,2017NE000015,23106.111709/2017-11,047.828.736-46,BRENO GIOVANNI ADAID\r\nCASTRO,PAGAMENTO DA FOLHA- 09/2017-692 // AUXILIO A\r...,0250022135,3390.20.01,AUXILIO A PESQUISADORES,154019,CENTRO DE APOIO AO\r\nDESENVOLV. TECNOLOGICO\r...,10,"4.200,00"
4,02/10/2017,2017OB806261,2017NE000015,23106.111714/2017-16,863.689.901-04,FABIO JACINTO BARRETO\r\nDE SOUZA,PAGAMENTO DA FOLHA- 09/2017-693 // AUXILIO A\r...,0250022135,3390.20.01,AUXILIO A PESQUISADORES,154019,CENTRO DE APOIO AO\r\nDESENVOLV. TECNOLOGICO\r...,10,"5.500,00"


### Limpar a coluna Fonte de Recursos (remover tudo que não for numero)

In [39]:
# Remover todos os caracteres e deixar somente os numeros
def remove_chars(element):
    return "".join(i for i in element if i.isdigit())

df_clean['Fonte de Recurso'] = df_clean['Fonte de Recurso'].astype(str)
df_clean['Fonte de Recurso'] = df_clean['Fonte de Recurso'].apply(remove_chars)

df_clean['Fonte de Recurso'].head()

0    0250154079
1    0250022135
2    0250022135
3    0250022135
4    0250022135
Name: Fonte de Recurso, dtype: object

### Limpar a coluna Pago (Remover tudo que não for numero ou ,)

In [40]:
def still_numbers_and_commas(element):
    return "".join(i for i in element if i.isdigit() or i == ',')

df_clean['Pago'] = df_clean['Pago'].astype(str)

df_clean['Pago'] = df_clean['Pago'].apply(still_numbers_and_commas)

df_clean['Pago'] = df_clean['Pago'].str.replace(',','.')

df_clean['Pago'].head()

0    39667.80
1     1810.00
2     3500.00
3     4200.00
4     5500.00
Name: Pago, dtype: object

### Conversão de Str para Float ** Problema: Não consigo converter!

In [41]:
# Está ocorrendo varios erros durante a conversão... Não consigo entender o pq
df_clean['Pago'] = df_clean['Pago'].astype(float64, errors = 'ignore')

df_clean['Pago'].head()

0    39667.80
1     1810.00
2     3500.00
3     4200.00
4     5500.00
Name: Pago, dtype: object

In [ ]:
# Este trecho de codigo mostra que em todas as linhas está ocorrendo este erro
# for i in df_clean['Pago']:
#     print(i)
#     try:
#         i = float(i)
#     except ValueError, e:
#         print "Error", e, " on number", i

### Criando um Panda Series com as informações agrupadas nos códigos da Fonte de Recurso

In [53]:
# Agrupando e somando as linhas com codigos iguais
table_gastos = df_clean.groupby('Fonte de Recurso')['Pago'].sum()

# Configurando modo de exibição de floats
pd.options.display.float_format = '${:,.2f}'.format

table_gastos.head()

Fonte de Recurso
                20.00449.00360.00615.0020.00200.00379.16379.16...
00112000000                     28.9020469.73555.9925927.18540.00
00250155512                                        3672.601292.77
00250262710                                  9472.182687.56500.00
006112000000                                              1500.00
Name: Pago, dtype: object

### Convertendo Seires para Dataframe

In [48]:
# Convertendo Series para Dataframe
df_gastos = pd.DataFrame(table_gastos).reset_index()

# Ordenando em ordem alfabética para facilitar visualização de descrições que podem ser agrupadas
df_gastos = df_gastos.sort_values(by='Fonte de Recurso')

df_gastos.head()

,Fonte de Recurso,Pago
0,,20.00449.00360.00615.0020.00200.00379.16379.16...
1,00112000000,28.9020469.73555.9925927.18540.00
2,00250155512,3672.601292.77
3,00250262710,9472.182687.56500.00
4,006112000000,1500.00


In [55]:
# Agrupando e somando as linhas com descrição iguais
df_gastos = df_gastos.reset_index()
df_gastos = df_gastos.groupby('Fonte de Recurso')['Pago'].sum()
df_gastos.head()

Fonte de Recurso
                20.00449.00360.00615.0020.00200.00379.16379.16...
00112000000                     28.9020469.73555.9925927.18540.00
00250155512                                        3672.601292.77
00250262710                                  9472.182687.56500.00
006112000000                                              1500.00
Name: Pago, dtype: object

### Salvando o resultado em um CSV

In [50]:
# Salvando o resultado em um csv
df_gastos.to_csv('../csv/parsed/Fonte_de_Recurso_X_Pago.csv')